# Connecting to econ data in Sagemaker Studio Lab with R 

[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/MarjorieRWillner/DisasterHack/blob/main/econ_data_extract_r.ipynb)

## FRED data

Short for Federal Reserve Economic Data, FRED is an online database consisting of hundreds of thousands of economic data time series from scores of national, international, public, and private sources. FRED, created and maintained by the Research Department at the Federal Reserve Bank of St. Louis, goes far beyond simply providing data: It combines data with a powerful mix of tools that help the user understand, interact with, display, and disseminate the data. In essence, FRED helps users tell their data stories. The purpose of this article is to guide the potential (or current) FRED user through the various aspects and tools of the database.

This is likely the easiest way to get data from U.S. statistical agencies like the Census and the BLS. In order to get this code to work you must obtain an api key from FRED: https://fredaccount.stlouisfed.org/apikeys

We will be using the fredr package: https://sboysel.github.io/fredr/index.html

In [6]:
# This code will gather the population of each US state over time

# install.packages("fredr")
# install.packages("tidyverse")
suppressWarnings(suppressMessages(library(fredr)))
suppressWarnings(suppressMessages(library(tidyverse)))

# your api key goes here
fredr_set_key("")

# the fredr series id we want is DCPOP, FLPOP, VAPOP, MDPOP, etc. We will create
# a list with all the states in the format of [ST]POP.
state_id <- c(map_chr(state.abb, ~ paste0(.x, "POP")), "DCPOP")

# We can now map over state_id and get the population for each state using the
# fredr() function
state_df <- map_dfr(
  state_id,
  ~ fredr(
    series_id = .x, 
    observation_start = as.Date("2020-01-01"), 
    observation_end = as.Date("2021-01-01")
    )
  )

glimpse(state_df)

Rows: 102
Columns: 5
$ date           <date> 2020-01-01, 2021-01-01, 2020-01-01, 2021-01-01, 2020-0…
$ series_id      <chr> "ALPOP", "ALPOP", "AKPOP", "AKPOP", "AZPOP", "AZPOP", "…
$ value          <dbl> 5024.803, 5039.877, 732.441, 732.673, 7177.986, 7276.31…
$ realtime_start <date> 2022-01-21, 2022-01-21, 2022-01-21, 2022-01-21, 2022-0…
$ realtime_end   <date> 2022-01-21, 2022-01-21, 2022-01-21, 2022-01-21, 2022-0…


***
## Census data

Connect to Census via the tidycensus package - tidycensus is an R package that allows users to interface with a select number of the US Census Bureau’s data APIs and return tidyverse-ready data frames, optionally with simple feature geometry included.

Accessing the Census directly allows for retrievel of ACS microdata files so that we can combine in a way that makes sense for our project. To get started working with tidycensus, users should load the package along with the tidyverse package, and set their Census API key. A key can be obtained from http://api.census.gov/data/key_signup.html.

NOTE: To install tidycensus in sagemake you need to run the following code in the terminal 

    conda install -n R -c conda-forge r-tidycensus

Reference: https://aws.amazon.com/blogs/machine-learning/creating-a-persistent-custom-r-environment-for-amazon-sagemaker/

In [ ]:
# you can run terminal commands in r via the system() function as shown here.
system(command = 'conda install -c r tidycensus --yes')

***
get_pums() returns some technical variables by default without the user needing to request them specifically. These include:

- SERIALNO: a serial number that uniquely identifies households in the sample;
- SPORDER: the order of the person in the household; when combined with SERIALNO, uniquely identifies a person;
- WGTP: the household weight;
- PWGTP: the person weight
- PUMA: Public Use Microdata Areas (PUMAs) are the smallest available geographies at which records are identifiable in the PUMS datasets. PUMAs are redrawn with each decennial US Census, and typically are home to 100,000-200,000 people. In large cities, a PUMA will represent a collection of nearby neighborhoods; in rural areas, it might represent several counties across a large area of a state.

It is a long-form dataset that organizes specific value codes by variable so you know what you can get. You'll use information in the var_code column to fetch variables, but pay attention to the var_label, val_code, val_label, and data_type columns.

In [2]:
suppressWarnings(suppressMessages(library("tidycensus")))
suppressWarnings(suppressMessages(library("tidyverse")))

# your api key goes here
census_api_key("", overwrite = TRUE, install = FALSE)

# we can get a list of all the variables that are available to us to choose from
pums_var <- pums_variables %>%
  filter(year == 2019, survey == "acs5") %>%
  arrange(var_label) 

# we now have a table with all the variables available to us
glimpse(pums_var)

To install your API key for use in future sessions, run this function with `install = TRUE`.



Rows: 5,277
Columns: 12
$ survey     <chr> "acs5", "acs5", "acs5", "acs5", "acs5", "acs5", "acs5", "ac…
$ year       <chr> "2019", "2019", "2019", "2019", "2019", "2019", "2019", "20…
$ var_code   <chr> "ENG", "ENG", "ENG", "ENG", "ENG", "FENGP", "FENGP", "ACCES…
$ var_label  <chr> "Ability to speak English", "Ability to speak English", "Ab…
$ data_type  <chr> "chr", "chr", "chr", "chr", "chr", "chr", "chr", "chr", "ch…
$ level      <chr> "person", "person", "person", "person", "person", "person",…
$ val_min    <chr> "b", "1", "2", "3", "4", "0", "1", "b", "1", "2", "3", "b",…
$ val_max    <chr> "b", "1", "2", "3", "4", "0", "1", "b", "1", "2", "3", "b",…
$ val_label  <chr> "N/A (less than 5 years old/speaks only English)", "Very we…
$ recode     <lgl> TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE,…
$ val_length <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 7, 7, 7, 7, 7,…
$ val_na     <dbl> 0, 0, 0, 0, 0, NA, NA, 0, 0, 0, 0, -1, -1, -1, NA, NA, NA, …


There are *more than 500* variables to choose from. Please reference the PUMS data dictionary located here: https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/. 

The most recent 5 year ACS data dictionary is here: https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMS_Data_Dictionary_2015-2019.pdf 

The most recent 1 year ACS data dictionary is here: https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMS_Data_Dictionary_2019.pdf

Once you know what you want you can use the get_pums() function to bring in all the microdata. 

**WARNING** Be careful here, Census ACS microdata is huge and can take a long time to retrieve if the request is not targetted well enough.

In [ ]:
# retrieve 5 year ACS microdata for CA. This is 480 MB and takes a few minutes to download.
state_pums <- get_pums(
  variables = c(
    "ACCESS", "ADJINC", "AGEP", "CIT", "CITWP", "DECADE", "ELEP", "ENG", "FINCP", "GASP", "GRNTP", "GRPIP", "HHT", "HINCP", "HISPEED", "MIGSP", "MRGP", 
      "MV",  "NAICSP", "NATIVITY", "NOP", "NPF", "OCPIP", "OCCP", "PINCP", "PERNP", "POBP", "POVPIP", "PUMA", "REGION", "RNTP", "SCHL", "SEX", "ST",
      "TAXAMT", "TEN", "WAGP", "WATP", "WGTP", "YOEP", "ACR", "NP", "BLD"
  ),
  state = "CA",
  survey = "acs5",
  recode = TRUE
)

glimpse(state_pums)

Getting data from the 2015-2019 5-year ACS Public Use Microdata Sample



Downloading: 480 MB     

***
Some of this microdata pertains to person level data and household level data. We need to split the data into these 2 categories and then apply adjustment factors.

In [1]:
# lets address the householf variables first
state_pums_hh <- state_pums %>%
    transmute(
        SERIALNO,
        NPF,
        WGTP,
        ADJHSG = as.numeric(ADJHSG),
        ADJINC = as.numeric(ADJINC),
        PUMA,
        ST,
        ST_label,
        ACCESS,
        ACCESS_label,
        ACR,
        ACR_label,
        BLD,
        BLD_label,
        ELEP = as.numeric(ELEP) * ADJHSG,
        FINCP = as.numeric(FINCP) * ADJINC,
        GASP = as.numeric(GASP) * ADJHSG,
        GRPIP,
        GRNTP = as.numeric(GRNTP) * ADJHSG,
        HHT,
        HHT_label,
        HINCP = as.numeric(HINCP) * ADJINC,
        HISPEED,
        HISPEED_label,
        MRGP = as.numeric(MRGP) * ADJHSG,
        MV,
        MV_label,
        NP,
        OCPIP,
        RNTP = as.numeric(RNTP) * ADJHSG,
        TAXAMT = as.numeric(TAXAMT) * ADJHSG,
        TEN,
        TEN_label,
        WATP = as.numeric(WATP) * ADJHSG
    ) %>%
    distinct()

# next we'll do person level data but first lets get a list of all the household level variables
state_pums_hh_colnames <- colnames(state_pums_hh)

# create person level data
state_pums_p <- state_pums %>%
    # remove the household level variables
    select(-one_of(state_pums_hh_colnames)) %>%
    # bring back in the weights and id variables
    bind_cols(
        state_pums %>%
            transmute(
                SERIALNO,
                NPF,
                ADJHSG = as.numeric(ADJHSG),
                ADJINC = as.numeric(ADJINC),
                PUMA,
                ST,
                ST_label
            )
    ) %>%
    # keep the person level variables and make adjustments
    transmute(
        SERIALNO,
        SPORDER,
        PWGTP,
        ADJHSG = ADJHSG/1000000,
        ADJINC = ADJINC/1000000,
        PUMA,
        ST,
        ST_label,
        AGEP,
        CIT,
        CIT_label,
        CITWP = lubridate::ymd(CITWP, truncated = 2L),
        DECADE,
        DECADE_label,
        ENG,
        ENG_label,
        MIGSP,
        MIGSP_label,
        NAICSP,
        NAICSP_label,
        NATIVITY,
        NATIVITY_label,
        NOP,
        NOP_label,
        OCCP,
        OCCP_label,
        PERNP = as.numeric(PERNP) * ADJINC,
        PINCP = as.numeric(PINCP) * ADJINC,
        POBP,
        POBP_label,
        POVPIP,
        SCHL,
        SCHL_label,
        SEX,
        SEX_label,
        WAGP = as.numeric(WAGP) * ADJINC,
        YOEP = lubridate::ymd(CITWP, truncated = 2L)
    )

ERROR: Error in state_pums %>% transmute(SERIALNO, NPF, WGTP, ADJHSG = as.numeric(ADJHSG), : could not find function "%>%"


With this cleaned up data we can start getting counts

In [ ]:
# get population counts by building units in structure
state_pums_hh %>%
    group_by(PUMA) %>%
    mutate(
        tot_pop = sum(WGTP)
    ) %>%
    group_by(PUMA, tot_pop, BLD_label) %>%
    summarize(
        BLD_tot_pop = sum(WGTP),
        BLD_per = round(BLD_tot_pop/tot_pop, 2)
    ) %>%
    distinct()